<a href="https://colab.research.google.com/github/miguelpuente/Analisis-Ventas-Python-Pandas/blob/eda/An%C3%A1lisis_de_datos_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de datos

## 1 - ETL (Extración, Transformación y Carga)

### 1. Extración (Extract)

In [1]:
from google.colab import drive
import pandas as pd
import chardet

# Montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/ventas.csv'
with open(ruta_archivo, 'rb') as f:
    result = chardet.detect(f.read())

try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=';', decimal=',')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')


Mounted at /content/drive
Extración de datos exitosa


### 2. Transformación (Transform)

In [ ]:
# Muestra las primeras 5 filas del DataFrame
df.head(10)

In [8]:
# Resumen de la información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   tienda      800 non-null    object        
 1   marca       800 non-null    object        
 2   tipo        800 non-null    object        
 3   genero      800 non-null    object        
 4   talla       800 non-null    object        
 5   color       800 non-null    object        
 6   categoria   800 non-null    object        
 7   precio      800 non-null    float64       
 8   fecha_hora  800 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 56.4+ KB


In [3]:
# Renombrar columnas para facilitar el acceso y evitar caracteres especiales
df.rename(columns ={
    'Tienda': 'tienda',
    'Marca': 'marca',
    'Tipo': 'tipo',
    'Gï¿½nero': 'genero',
    'Talla': 'talla',
    'Color': 'color',
    'Categoria': 'categoria',
    'precio de venta': 'precio',
    'Fecha': 'fecha',
    'Hora': 'hora'
}, inplace=True)

# Convertí la columna 'precio' a numérica
df['precio'] = df['precio'].astype(str)
df['precio'] = df['precio'].str.replace(',', '.', regex=True).astype(float)

# Limpiar y unificar los datos de la columna 'genero'
df['genero'] = df['genero'].str.replace('Masculino', 'M', regex=False).str.replace('Femenino', 'F', regex=False).str.replace('Usinex', 'U', regex=False)

# Crear una columna de fecha y hora combinada y convertila al tipo datatime
df['fecha_hora'] = pd.to_datetime(df['fecha'] + ' ' + df['hora'], format='%d/%m/%Y %H:%M:%S')

# Eliminar las columnas originales de fecha y hora
df.drop(columns=['fecha', 'hora'], inplace=True)

print("Transformación de datos exitosa")


Transformación de datos exitosa


### 3. Carga (Load)

In [ ]:
# definir la ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/ventas_transformadas.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV en Drive
df.to_csv(ruta_salida, index=False, sep=',')

print(f'Archivo CSV guardado en: {ruta_salida}')

## 2 - EDA (Análisis Exploratorio de datos)

In [ ]:
# Cuanto facturó cada tienda.
facturacion_ordenada = df.groupby(['tienda', 'marca'])['precio'].sum().reset_index()

top = facturacion_ordenada.sort_values(by='precio', ascending=False)

print(top.head(3))

In [ ]:
# Filtrado condicional -- Boolean Indexing

# 1 - Definir la condición: categoria es Pantalon & color es Negro
condicion = (df['categoria'] == 'Pantalon') & (df['color'] == 'Negro')

# 2 - Aplicar la condición al DataFrame
df_filtrado = df[condicion]

# 3 - Responder la pregunta
conteo_transacciones = df_filtrado['precio'].count()
facturacion_total = df_filtrado['precio'].sum()

print(f'Cantidad de pantalones negros vendidos : {conteo_transacciones} por un monto de {facturacion_total} ')



In [ ]:
# Extracción de Componentes de Fecha y Hora (.dt)

# Pregunta de Negocio: ¿Qué día de la semana (Lunes=0 a Domingo=6) fue el más rentable en términos de facturación total?

# Atributo .dt	Valor Devuelto
# .dt.dayofweek	Número del día de la semana (0 a 6).
# .dt.day_name()	Nombre del día de la semana.
# .dt.month	Número del mes (1 a 12).

# 1. Extraer el día de la semana (como nombre)
df['dia_semana'] = df['fecha_hora'].dt.day_name()

# 2. Agrupar la facturación por el nuevo día de la semana
rentabilidad_por_dia = df.groupby('dia_semana')['precio'].sum().reset_index(name='facturacion_total')

# 3. Ordenar para encontrar el día más rentable
dia_mas_rentable = rentabilidad_por_dia.sort_values(by='facturacion_total', ascending=False)

print("\n--- Análisis Temporal: Facturación por Día de la Semana ---")
print(dia_mas_rentable)

In [ ]:
# Tablas Dinámicas (Pivot Tables)

# Pregunta de Negocio: ¿Cuál es el precio promedio de los productos, desglosado por tienda (en filas)
#y por categoría (en columnas)?

# Parámetro	Función
# index	La(s) columna(s) que se usarán como filas (eje X).
# columns	La(s) columna(s) que se usarán como columnas (eje Y).
# values	La columna sobre la cual se aplicará la agregación.
# aggfunc	La función de agregación a aplicar ('mean', 'sum', 'count', etc.).

# 1. Usar el método pivot_table
tabla_promedio_tienda_categoria = pd.pivot_table(
    df,
    values='precio',     # Queremos el promedio del 'precio'
    index='tienda',      # Desglosado en filas por 'tienda'
    columns='categoria', # Desglosado en columnas por 'categoria'
    aggfunc='count'       # La función de agregación es el promedio
)

#print("\n--- Tabla Dinámica: Precio Promedio por Tienda y Categoría ---")
print(tabla_promedio_tienda_categoria)
